In [0]:
import zipfile
import io

ZIP_SOURCE_PATH = "abfss://divvycontainer@divvystorage1.dfs.core.windows.net/trips"
UNZIPPED_DEST_PATH = "abfss://divvycontainer@divvystorage1.dfs.core.windows.net/raw"
ARCHIVE_PATH = "abfss://divvycontainer@divvystorage1.dfs.core.windows.net/trips_archive"

dbutils.fs.mkdirs(ARCHIVE_PATH)

def extract_and_write_zip_files():
    zip_files = [
        f.path for f in dbutils.fs.ls(ZIP_SOURCE_PATH)
        if f.name.endswith('.zip')
    ]
    for zip_path in zip_files:
        # Read ZIP file as binary
        binary_df = spark.read.format("binaryFile").load(zip_path)
        zip_bytes = binary_df.collect()[0]['content']
        with zipfile.ZipFile(io.BytesIO(zip_bytes)) as zip_ref:
            for contained_file in zip_ref.namelist():
                if contained_file.lower().endswith('.csv'):
                    csv_bytes = zip_ref.read(contained_file)
                    output_file_path = f"{UNZIPPED_DEST_PATH}/{contained_file}"
                    dbutils.fs.put(
                        output_file_path,
                        csv_bytes.decode("utf-8", errors='ignore'),
                        overwrite=True
                    )
        # Move processed zip to archive
        file_name = zip_path.split('/')[-1]
        archive_target = f"{ARCHIVE_PATH}/{file_name}"
        dbutils.fs.mv(zip_path, archive_target)

extract_and_write_zip_files()